In [1]:
import numpy as np
import os
import pickle
from time import time
from datetime import datetime

In [2]:
import pulse2percept as p2p
import p2pspatial

/home/mbeyeler/anaconda3/lib/python3.5/site-packages/skvideo/__init__.py:356: UserWarning: avconv/avprobe not found in path: 
  warnings.warn("avconv/avprobe not found in path: " + str(path), UserWarning)
2018-01-09 15:33:15,884 [pulse2percept] [INFO] Welcome to pulse2percept


In [3]:
subject = '12-005'
amplitude = 2.0
electrodes = None
random_state = 42
n_folds = 5

In [4]:
t_start = time()
now = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
filename = 'crossval-swarm_%s_%s.pickle' % (subject, now)
print(filename)

crossval-swarm_12-005_2018-01-09_15-33-16.pickle


In [5]:
rootfolder = os.path.join(os.environ['SECOND_SIGHT_DATA'], 'shape')
X, y = p2pspatial.load_data(rootfolder, subject=subject, electrodes=electrodes,
                            amplitude=amplitude, random_state=random_state,
                            single_stim=True, verbose=True)
print(X.shape, y.shape)
if len(X) == 0:
    raise ValueError('no data found')

Found 55 samples in /home/mbeyeler/data/secondsight/shape/51-009/20100217/2xTh 20Hz 0.45ms 250ms
Found 60 samples in /home/mbeyeler/data/secondsight/shape/51-009/20091028/VLevel_31_6Hz_Pilot
Found 50 samples in /home/mbeyeler/data/secondsight/shape/51-009/20100203/2xTh 20Hz 0.45ms 250ms
Found 80 samples in /home/mbeyeler/data/secondsight/shape/51-009/20100504/1.5xTh 2xTh 20Hz 0.45ms Week1_VCF2
Found 50 samples in /home/mbeyeler/data/secondsight/shape/51-009/20100504/2xTh 20Hz 0.45ms Week2_VCF1
Found 10 samples in /home/mbeyeler/data/secondsight/shape/51-009/20100504/2xTh 20Hz 0.45ms Week1_VCF1
Found 45 samples in /home/mbeyeler/data/secondsight/shape/51-009/20100526/2xTh 20Hz 0.45ms_Week4_VCF1
Found 35 samples in /home/mbeyeler/data/secondsight/shape/51-009/20100526/2xTh 20Hz 0.45ms_Week3_VCF1
Found 45 samples in /home/mbeyeler/data/secondsight/shape/51-009/20100526/2xTh 20Hz 0.45ms_Week4_VCF3
Found 35 samples in /home/mbeyeler/data/secondsight/shape/51-009/20100526/2xTh 20Hz 0.45ms_We

Could not parse row: 20100526_112217_trial9 blank
Could not parse row: 20100611_114004_trial87 blank
Could not parse row: 20100819_094916_trial14 blank
Could not parse row: 20100611_102522_trial62 blank
Could not parse row: 20100611_102522_trial58 blank
Could not parse row: 20100203_171247_trial25 blank
Could not parse row: 20100526_102942_trial13 blank
Could not parse row: 20100611_102522_trial36 blank
Could not parse row: 20100526_105201_trial33 blank
Could not parse row: 20100611_102522_trial51 blank
Could not parse row: 20100526_112933_trial31 blank
Could not parse row: 20100611_102522_trial2 blank
Could not parse row: 20100819_093114_trial6 blank
Could not parse row: 20100210_144555_trial24 blank
Could not parse row: 20100526_101705_trial2 blank
Could not parse row: 20100526_101705_trial13 blank
Could not parse row: 20100812_095057_trial34 blank
Could not parse row: 20100526_110247_trial6 blank
Could not parse row: 20100526_111115_trial1 blank
Could not parse row: 20100526_111115_

Could not parse row: 20100506_100449_trial31 blank
Could not parse row: 20100812_095057_trial2 blank
Could not parse row: 20100611_102522_trial53 blank
Could not parse row: 20100526_111115_trial35 blank
Could not parse row: 20100611_113015_trial68 blank
Could not parse row: 20100429_110702_trial33 blank
Could not parse row: 20100526_104243_trial29 blank
Could not parse row: 20100611_114004_trial3 blank
Could not parse row: 20100611_114004_trial15 blank
Could not parse row: 20100526_110247_trial28 blank
Could not parse row: 20100611_102522_trial21 blank
Could not parse row: 20100611_102522_trial69 blank
Could not parse row: 20100210_144555_trial17 blank
Could not parse row: 20100526_102942_trial20 blank
Could not parse row: 20100611_113015_trial75 blank
Could not parse row: 20100526_105201_trial24 blank
Could not parse row: 20100526_105201_trial15 blank
Could not parse row: 20100506_100449_trial28 blank
Could not parse row: 20100611_103836_trial28 blank
Could not parse row: 20100526_111

Could not parse row: 20100506_100449_trial24 blank
Could not parse row: 20100526_101705_trial1 blank
Could not parse row: 20100506_100449_trial20 blank
Could not parse row: 20100506_102431_trial20 blank
Could not parse row: 20100210_144555_trial15 blank
Could not parse row: 20100210_144555_trial18 blank
Could not parse row: 20100429_105204_trial29 blank
Could not parse row: 20100505_101436_trial14 blank
Could not parse row: 20100210_144555_trial38 blank
Could not parse row: 20100505_104011_trial41 blank
Could not parse row: 20100526_110247_trial31 blank
Could not parse row: 20100611_103836_trial55 blank
Could not parse row: 20100526_110247_trial19 blank
Could not parse row: 20100506_105759_trial1 blank
Could not parse row: 20100611_102522_trial77 blank
Could not parse row: 20100611_102522_trial62 blank
Could not parse row: 20100506_105119_trial8 blank
Could not parse row: 20100506_100449_trial27 blank
Could not parse row: 20100611_103836_trial43 blank
Could not parse row: 20100526_1111

In [6]:
model_params = {'sampling': 200,
                'csmode': 'gaussian',
                'sensitivity_rule': 'decay',
                'thresh': 1.0}
regressor = p2pspatial.SpatialModelRegressor(**model_params)

In [7]:
search_params = {'decay_const': (0.001, 10),
                 'cswidth': (10, 1000),
                 'implant_x': (-1500, 1500),
                 'implant_y': (-1000, 1000),
                 'implant_rot': np.deg2rad((-75, -15))}
pso_options = {'max_iter': 100,
               'min_func': 1e-2}
pso = p2pspatial.model_selection.ParticleSwarmOptimizer(
    regressor, search_params, **pso_options
)

In [8]:
fit_params = {'loc_od_x': 15.5,
              'loc_od_y': 1.5,
              'use_persp_trafo': True,
              'use_ofl': True,
              'scoring_weights': {'area': 0.001,
                                  'orientation': 100.0,
                                  'major_axis_length': 0.1,
                                  'minor_axis_length': 0.1}}
y_test, y_pred, best_params = p2pspatial.model_selection.crossval_predict(
    pso, X, y, fit_params=fit_params, n_folds=n_folds)

No constraints given.
implant (x, y): (1235.79, 471.01), rot: -0.533183
Set loc_od: 15.5 1.5 decay_const: 6.55744200688 sensitivity_rule: decay thresh: 1.0


KeyboardInterrupt: 

In [ ]:
for k, v in search_params.items():
    setattr(regressor, k, v[0])

In [ ]:
regressor.fit(X, y=y, **fit_params)

In [ ]:
regressor.score(X, y)

In [ ]:
print("Done in %.3fs" % (time() - t_start))

In [ ]:
specifics = {'subject': subject,
             'amplitude': amplitude,
             'electrodes': electrodes,
             'n_folds': n_folds,
             'regressor': regressor,
             'optimizer': pso,
             'model_params': model_params,
             'search_params': search_params,
             'fit_params': fit_params,
             'now': now,
             'random_state': random_state}
pickle.dump((y_test, y_pred, best_params, specifics), open(filename, 'wb'))
print('Dumped data to %s' % filename)